# Capstone Project - The Battle of the Neighborhoods (Week 2)
# Bogota D.C- Colombia
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

# Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Argentinian restaurant** in **Bogota**, Colombia.

because the success project is depending of cultural zones that restaurant can obtain good profit in this city and it can get potential customers.

we will use the datascience tools such as python and foursquare to find the best location and we can get potential customers. 

# Data <a name="data"></a>

For this Capstone I am going to use the following data source:

Original Data source: Georeferencia puntual por localidad

Url:https://bogota-laburbano.opendatasoft.com/explore/dataset/georeferencia-puntual-por-localidad/table/?flg=es

This data source was modified because the CSV setting is different from the Python requirements, and the values were separated with the semicolon, on the other hand, some titles translated into English for I can connect with  API foursquare and shows the venues.

Finally, the file with modified data is :
https://raw.githubusercontent.com/camiolo85/Coursera-Capstone/master/Data_bogota2.csv


This data source contains the following fields :

Neighborhoods(Localidades)
Code(Número de Localidad) 
Longitude
Latitude

With this data, API foursquare can find the venues and start with analysis.

## Identifying Neighborhoods

In [1]:
import requests
import pandas as pd
import numpy as np # 
import random #


!pip install geopy
from geopy.geocoders import Nominatim 


from IPython.display import Image 
from IPython.core.display import HTML 
    
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 79 kB 9.5 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=b6970245fc9ebe1a810f1e43ca628e63df69e5f35718d564ada02ac75f159981
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
Folium installed
Libraries imported.


In [17]:
url="https://raw.githubusercontent.com/camiolo85/Coursera-Capstone/master/Data_bogota2.csv"
df_bog = pd.read_csv(url)
df_bog.head(22)

,CODE,NEIGHBORHOOD,LONGITUDE,LATITUDE
0,2,CHAPINERO,-74.467,4.657
1,6,TUNJUELITO,-74.107,4.588
2,15,ANTONIO NARIÑO,-74.101,4.549
3,16,PUENTE ARANDA,-74.123,4.615
4,1,USAQUEN,-74.031,4.749
5,0,BOGOTA,-74.082,4.610
6,7,BOSA,-74.195,4.631
7,19,CIUDAD BOLIVAR,-74.154,4.507
8,18,RAFAEL URIBE URIBE,-74.116,4.565
9,8,KENNEDY,-74.157,4.627


In [3]:
df_bog.shape

(21, 4)

## Connecting with Foursquare to view venues

In [4]:
CLIENT_ID = 'XLP0OKZ4MK2R2FJMKSFLYUE4ZZI4BFZBI24431A4QSNQOF5K' 
CLIENT_SECRET = 'GLN35DWF50GIZXFDMSJ0WITMN35O0HBV4I54MZQIMBSHYNUB' 
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XLP0OKZ4MK2R2FJMKSFLYUE4ZZI4BFZBI24431A4QSNQOF5K
CLIENT_SECRET:GLN35DWF50GIZXFDMSJ0WITMN35O0HBV4I54MZQIMBSHYNUB


In [5]:
df_bog.loc[0, 'NEIGHBORHOOD']

'CHAPINERO'

In [6]:
NEIGHBORHOOD_LATITUDE = df_bog.loc[0, 'LATITUDE'] 
NEIGHBORHOOD_LONGITUDE = df_bog.loc[0, 'LONGITUDE']

NEIGHBORHOOD = df_bog.loc[3, 'NEIGHBORHOOD'] 

print('Latitude and longitude values of {} are {}, {}.'.format(NEIGHBORHOOD, 
                                                               NEIGHBORHOOD_LATITUDE , 
                                                               NEIGHBORHOOD_LONGITUDE))

Latitude and longitude values of PUENTE ARANDA are 4.657, -74.467.


In [7]:
limit = 100
radius = 3000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    NEIGHBORHOOD_LATITUDE,
    NEIGHBORHOOD_LONGITUDE,
    radius,
    limit)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=XLP0OKZ4MK2R2FJMKSFLYUE4ZZI4BFZBI24431A4QSNQOF5K&client_secret=GLN35DWF50GIZXFDMSJ0WITMN35O0HBV4I54MZQIMBSHYNUB&v=20180605&ll=4.657,-74.467&radius=3000&limit=100'

In [8]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60035a37879fa652d7000319'},
 'response': {'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 4.684000027000027,
    'lng': -74.43996111898626},
   'sw': {'lat': 4.629999972999973, 'lng': -74.49403888101374}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4f6688c7e4b0656126b90adb',
       'name': 'Parque Principal La Mesa',
       'location': {'lat': 4.6321044560427325,
        'lng': -74.46290946218203,
        'labeledLatLngs': [{'label': 'display',
          'lat': 4.6321044560427325,
          'lng': -74.46290946218203}],
        'distance': 2808,
        'cc': 'CO',
        'city': 'La Mesa',
        

In [9]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]



nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Parque Principal La Mesa,Park,4.632104,-74.462909
1,Roa Velez,Bakery,4.630448,-74.462692
2,Ecoparque Macadamia Bosque Aventura,Trail,4.644816,-74.448860
3,Cosechas,Juice Bar,4.633350,-74.460379
4,Hotel Bogotá (LA MESA),Hotel,4.632199,-74.463757


In [11]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

6 venues were returned by Foursquare.


In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
Bogota_venues = getNearbyVenues(names=df_bog['NEIGHBORHOOD'],
                                   latitudes=df_bog['LATITUDE'],
                                   longitudes=df_bog['LONGITUDE']
                                  )

CHAPINERO
TUNJUELITO
ANTONIO NARIÑO
PUENTE ARANDA
USAQUEN
BOGOTA
BOSA
CIUDAD BOLIVAR
RAFAEL URIBE URIBE
KENNEDY
BARRIOS UNIDOS
ENGATIVA
SUMAPAZ
TEUSAQUILLO
LA CANDELARIA
SANTA FE
SUBA
FONTIBON
LOS MARTIRES
SAN CRISTOBAL
USME


In [18]:
print(Bogota_venues.shape)
Bogota_venues.head(20)

(151, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,TUNJUELITO,4.588,-74.107,Bar Pescadero Del Restrepo,4.587761,-74.107269,Seafood Restaurant
1,TUNJUELITO,4.588,-74.107,Sopas y carnes del Restrepo,4.587374,-74.103828,BBQ Joint
2,TUNJUELITO,4.588,-74.107,Asadero El Gran Rodeo,4.590334,-74.107811,Food Court
3,TUNJUELITO,4.588,-74.107,Donde Canta La Rana,4.588010,-74.106977,BBQ Joint
4,TUNJUELITO,4.588,-74.107,La mojarra roja,4.587492,-74.103530,Seafood Restaurant
5,TUNJUELITO,4.588,-74.107,Exito Centenario,4.586587,-74.110521,Grocery Store
6,TUNJUELITO,4.588,-74.107,Moto Cervantes,4.590890,-74.110298,Motorcycle Shop
7,ANTONIO NARIÑO,4.549,-74.101,Ikaii Tea,4.550597,-74.100110,Bubble Tea Shop
8,ANTONIO NARIÑO,4.549,-74.101,Parque Natural Entrenuves,4.551464,-74.101120,Campground
9,ANTONIO NARIÑO,4.549,-74.101,Heladería y Comidas Rápidas Gexx,4.551784,-74.098755,Burger Joint


# Methodology

In this project, basically, we took all data of foursquare that it can show all venues in the city, and we determine the potential districts(Localidades)
to open an international restaurant due to the restaurant success depends on we can get customers with enough resources to pay products.

On the other hand, we can segment the potential customers because Bogota has a social organization called classification(Estratos)
and it can determine the social class, salary, and education level in the neighborhoods.

Finally the foursquare capacity can help us to find the best districts and we ensure the restaurant success.

# Analysis

For this case we create an filter with foursquare data where it shows all Latin American Restaurant in Bogota, and we determine
the potencial districts(localidades), according with the foursquare outcomes.

We can see the potencial districts in the following line code:

## Filtering For Latin American Restaurant

In [23]:
Bogota_restaurant=Bogota_venues[Bogota_venues['Venue Category']\
                                .str.contains('Latin American Restaurant')].reset_index(drop=True)
Bogota_restaurant.index = np.arange(1, len(Bogota_restaurant)+1)
Bogota_restaurant.shape 
Bogota_restaurant.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,TEUSAQUILLO,4.645,-74.094,Chuleta y Champus,4.643392,-74.090860,Latin American Restaurant
2,TEUSAQUILLO,4.645,-74.094,Café y Aroma,4.648641,-74.093963,Latin American Restaurant
3,LA CANDELARIA,4.594,-74.074,El Son de los Grillos,4.595569,-74.072934,Latin American Restaurant
4,LA CANDELARIA,4.594,-74.074,La Puerta de la Catedral,4.597848,-74.074611,Latin American Restaurant
5,LA CANDELARIA,4.594,-74.074,Mama Lupe,4.597787,-74.074597,Latin American Restaurant
6,FONTIBON,4.683,-74.148,Piqueteadero Guadalajara,4.680663,-74.149130,Latin American Restaurant


According to the query, we can find four districts:

Puente Aranda
Teusaquillo
Candelaria
Fontibon

it has excellent restaurant offers and location in the city, because in those districts 
work a lot of people and the propose can be successful.

In [24]:
map_restaurants_bog= folium.Map(location=[4.60971, -74.08175], zoom_start=11)
from folium.plugins import HeatMap

Neighborhood1 = ['PUENTE ARANDA', 'TEUSAQUILLO', 'LA CANDELARIA', 'FONTIBON'] 
x= np.arange(len(Neighborhood1))

rainbow = ['#00ff00', '#ff00ff', '#0000ff', '#ffa500', '#ff0000']

for lat, lon, poi, Neigh in zip(Bogota_restaurant['Venue Latitude'], Bogota_restaurant['Venue Longitude'], Bogota_restaurant['Venue Category'], Bogota_restaurant['Neighborhood']):
    label = folium.Popup(str(poi) + ' Neigh ' + str(Neigh), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color=rainbow[Neighborhood1.index(Neigh)-1],
        fill=True,
        fill_color=rainbow[Neighborhood1.index(Neigh)-1],
        fill_opacity=0.7).add_to(map_restaurants_bog)
       
map_restaurants_bog

However we create a second query and it can show us the location of the argentinian restaurants in Bogota:

In [21]:
Bogota_restaurant_tar=Bogota_venues[Bogota_venues['Venue Category']\
                                .str.contains('Argentinian Restaurant')].reset_index(drop=True)
Bogota_restaurant_tar.index = np.arange(1, len(Bogota_restaurant_tar)+1)
Bogota_restaurant_tar.shape 
Bogota_restaurant_tar.head(20)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,LA CANDELARIA,4.594,-74.074,Restaurante Rescoldos,4.595584,-74.073180,Argentinian Restaurant
2,LA CANDELARIA,4.594,-74.074,Mi Viejo,4.597425,-74.073957,Argentinian Restaurant
3,LA CANDELARIA,4.594,-74.074,Patagonia,4.596873,-74.074994,Argentinian Restaurant


In [25]:
map_restaurants = folium.Map(location=[4.60971, -74.08175], zoom_start=11)
from folium.plugins import HeatMap

Neighborhood = ['PUENTE ARANDA', 'TEUSAQUILLO', 'LA CANDELARIA', 'FONTIBON'] 
x= np.arange(len(Neighborhood))

rainbow = ['#00ff00', '#ff00ff', '#0000ff', '#ffa500', '#ff0000']

for lat, lon, poi, Neigh in zip(Bogota_restaurant_tar['Venue Latitude'], Bogota_restaurant_tar['Venue Longitude'], Bogota_restaurant_tar['Venue Category'], Bogota_restaurant_tar['Neighborhood']):
    label = folium.Popup(str(poi) + ' Neigh ' + str(Neigh), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color=rainbow[Neighborhood.index(Neigh)-1],
        fill=True,
        fill_color=rainbow[Neighborhood.index(Neigh)-1],
        fill_opacity=0.7).add_to(map_restaurants)
       
map_restaurants

## Candelaria and Teusaquillo

This district is well known as the historical center of Bogota, it becomes the first option to visit for locals and foreign tourists
and it offers a wide cultural offer and differents activities to discover the Colombian culture through our history.

this location can find international offers such as Italian and Latin American restaurants and people can access easily bank services
such as money exchanges and withdrawing through ATM banks and the tourist can host in expensive hotels or cheap hostel with excellent
commodities.

On the other hand, teusaquillo is another district can offer wide cultural offer and international restaurants and it has a lot
universities in its neighborhoods, the social classification is good and it can be the other location to this restaurant.


# Results and Discussion

Definitely, Foursquare give us the enough information to complete the analysis, therefore 
we can say that the best district to open argentinian restaurant is Candelaria because currently this district
is recognized has a multicultural districts and its neighborhoods are close from state building and banks.

Additionality the first option to ride in Bogota is the historical center and it has the best highways to arrive to airport and 
move through the city, and we can get the potencial customers and obtain the best profit in our business.

Lastly we can show teusaquillo district as a second option to open Argentinian restaurant because this location does not have a argentinian
restaurants however it has not the same comercial level if we compared with candelaria district due to it is a residencial districts and 
it depends on the university students and office workers.

# Conclusion

During our analysis, we can find the districts with potential for open Argentinian Restaurants and we can make a segmentation with Foursquare data
the results show us the main venues in the city and analyze social and economic aspects that it can help to determine the best location for the business

According to data, we offer two options to make the opening restaurant, keeping in mind the cultural offer and the relevancy in the Bogota Economy that it is an increase
the tourist portfolio.

Finally, the stakeholders can make decisions with an international approach or local approach.